# Bitcoin - Complex Graph Analysis

## Steps
1. [Parse blockchain](#Parse-blockchain-using-btcgraph.py)
2. [Data preparation (BigQuery)](#2.-Data-preparation-(BigQuery))  
    2.1. [Upload data](#2.1.-Upload-data-to-Google-Big-Query)  
    2.2. [Create new indices](#2.2.-Create-indices-for-every-address)  
    2.3. [Build edge list](#2.3.-Build-edge-list-with-timestamps-using-the-new-indices)  
    2.4. [Export data to GCS](#2.4.-Export-dataset-to-Google-Cloud-Storage)  
    2.5. [Download edge list from GCS](#2.5.-Download-edge-list-from-google-cloud-as-csv-(without-timestamp))
3. [Graph construction](#3.-Build-Graph-using-NetworKit)
4. [Analysis](#4.-Analysis)  
    4.1. The graph object  
    4.2. Connected Components  
    4.3. [](#)

***

## 1. Parse blockchain using btcgraph.py

In [5]:
%run run.py -loc data -raw 1 -wt 1

Starting btc graph version 0.1.0 with the following arguments:
current wd:       /home/nero/python/wu/btc/python-bitcoin-graph
startfile:        blk00000.dat 
endfile:          deactivated
starttx:          deactivated
endtx:            deactivated
endts:            deactivated
blklocation:      data         
format:           deactivated
rawedges:         activated
withts:           activated
googlebigquery:   deactivated

Initializing...
[########################################]

10:56:40  -  New BtcGraph initialized
10:56:41  -  Start building...
10:56:41  -  Block File # 0/4
10:56:41  -  Processing data/blk00000.dat
10:57:09  -  Graph has       18,216,888 bytes
10:57:09  -  Graph has               17 mb
10:57:09  -  ---  -------   ----------------------------
10:57:09  -  -->  23.2 GB   total memory
10:57:09  -  -->  16.6 GB   of memory available
10:57:09  -  -->   5.7 GB   memory used
10:57:09  -  -->   0.4 GB   memory used by this process
10:57:09  -  -->  28.1  %   of memory us

***

## 2. Data preparation (BigQuery)
   * Fetch distinct addresses 
   * Create index ID for each address
   * Create edge list from transactions with new indexes

In [47]:
import os 
from google.cloud import storage, bigquery

credentials_path = ".gcpkey/wu-btcgraph.json"
bucket_name      = "wu-bitcoin"
file_name        = "btc"
target_folder    = "graph"
bq_project       = "wu-btcgraph"
bq_dataset       = "btc"
bq_table         = "bitcoin_transactions"

if not os.path.isdir(f'./{target_folder}'):
    os.makedirs(f'./{target_folder}')

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path

### 2.1. Upload data to Google Big Query 

In [16]:
%run run.py -gbq 1

startfile:        deactivated
endfile:          deactivated
starttx:          deactivated
endtx:            deactivated
endts:            deactivated
blklocation:      deactivated
format:           deactivated
rawedges:         deactivated
withts:           deactivated
googlebigquery:   activated

Initializing...
[########################################]

raw_blk_0.csv      successfully uploaded   
raw_blk_1.csv      successfully uploaded   
raw_blk_2.csv      successfully uploaded   
[###]
-----------------------------------------


### 2.2. Create indices for every address

In [7]:
# Code taken from the official docs
# https://cloud.google.com/bigquery/docs/quickstarts/quickstart-client-libraries

client = bigquery.Client()

query = """
    CREATE OR REPLACE TABLE `{0}.{1}.addresses` AS
        SELECT
          ROW_NUMBER() OVER() - 1 id, addr FROM 
          (
            SELECT DISTINCT addr FROM 
                (
                SELECT `to` AS addr FROM `{0}.{1}.{2}`
                UNION ALL
                SELECT `from` AS addr FROM `{0}.{1}.{2}` 
                )
              ORDER BY addr 
            ) 
            ORDER BY id
        """.format(bq_project,bq_dataset,bq_table)

query_job = client.query(query)
query_job.result()

### 2.3. Build edge list with timestamps using the new indices

In [4]:
try: client
except: client = bigquery.Client()
query = """
        CREATE OR REPLACE TABLE `{0}.{1}.graph_with_ts` AS
            SELECT  A.ts, B.from_id, C.to_id FROM 
            (
              (SELECT * FROM `{0}.{1}.{2}`) A
              LEFT JOIN 
              (SELECT id AS from_id, addr FROM `{0}.{1}.addresses`) B ON A.from = B.addr
              LEFT JOIN 
              (SELECT id AS to_id, addr FROM `{0}.{1}.addresses`) C ON A.to = C.addr
            )
        """.format(bq_project,bq_dataset,bq_table)

query_job = client.query(query)
query_job.result()

### 2.4. Export dataset to Google Cloud Storage

In [7]:
# Code taken from the official docs
# https://cloud.google.com/bigquery/docs/samples/bigquery-extract-table

destination_uri = "gs://{}/{}_*.csv".format(bucket_name, file_name)
dataset_ref = bigquery.DatasetReference(bq_project, bq_dataset)
table_ref = dataset_ref.table("graph_with_ts")

extract_job = client.extract_table(
    table_ref,
    destination_uri,
    # Location must match that of the source table.
    location="US",
)  # API request
extract_job.result()  # Waits for job to complete.

print(
    "Exported {}:{}.{} to {}".format(bq_project, bq_dataset, bq_table, destination_uri)
)

NameError: name 'project' is not defined

### 2.5. Download edge list from google cloud as csv (without timestamp)

In [7]:
storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)
file_names = map(lambda x: str(x).split(",")[1].strip(), list(bucket.list_blobs()))
for fn in file_names:
    if fn.startswith("{}_".format(file_name)):
        blob = bucket.blob(fn)
        blob.download_to_filename(target_folder + "/" + fn)
        print(f"{bucket_name}.{fn} copied to {target_folder}/{fn}")

wu-bitcoin.btc_000000000000.csv copied to graph/btc_000000000000.csv
wu-bitcoin.btc_000000000001.csv copied to graph/btc_000000000001.csv


KeyboardInterrupt: 

***

## 3. Build Graph using NetworKit

In [4]:
target_folder = "testfolder"
file_location = target_folder
files = os.listdir(file_location)
files.remove("output")
G = graph.Graph() #graphio.EdgeListReader(',', 0, directed=True).read(file_location+"/"+files[0])
print("-------")
for file in range(21):
    print(file)
    #nr = int(re.search("[1-9].[0-9]+", file).group(0))
    with open(file_location+"/"+"testf"+str(file)+".csv", "r") as csvfile:
        spamreader = csv.reader(csvfile)
        for u, v in spamreader:
            G.addEdge(int(u), int(v), addMissing=True)
            


-------
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [6]:
G.numberOfEdges()

210789327

In [1]:
from networkit import *
import csv

In [41]:
counter = 2
for j in range(12,35):
    with open("testfolder/testf{}.csv".format(counter), "w") as csvfile:
        spamwriter = csv.writer(csvfile)
        
        for i in range(int("{}0000000".format(j))-10000000,int("{}0000000".format(j))):
            spamwriter.writerow([i, i+1])
    counter += 1
        

KeyboardInterrupt: 

In [25]:
G = graph.Graph()
G.addEdge??

***

## 4. Analysis

In [21]:
%matplotlib inline
import matplotlib.pyplot as plt
import networkit as nk

### 4.1. The graph object

In [4]:
nodes = G.numberOfNodes()
edges = G.numberOfEdges()
print(f"G has {nodes} nodes and {edges} edges")

'wu-btcgraph.json'

### 4.2. Connected components

In [2]:
cc = nk.components.ConnectedComponents(G)
cc.run()
print("number of components ", cc.numberOfComponents())

### 4.3. Degree distribution

In [5]:
dd = sorted(nk.centrality.DegreeCentrality(G).run().scores(), reverse=True)
plt.xscale("log")
plt.xlabel("degree")
plt.yscale("log")
plt.ylabel("number of nodes")
plt.plot(dd)
plt.show()

In [17]:
try:
    import powerlaw
    fit = powerlaw.Fit(dd)
except ImportError:
    print ("Module powerlaw could not be loaded")

'/home/nero/python/wu/btc/python-bitcoin-graph'

In [6]:
bucket = storage_client.bucket("wu-bitcoin")

In [9]:
blob = bucket.blob("btc.csv")
blob.download_to_filename("btcttttttttt.csv")

In [20]:
from networkit import *

In [18]:
g = graph.Graph()

In [19]:
g.numberOfEdges()

0

In [22]:
import pandas as pd

In [25]:
pd.DataFrame([(1,2,3),(3,4,5)], columns=["ts", "from", "to"])

,ts,from,to
0,1,2,3
1,3,4,5


In [4]:
"""
    CREATE OR REPLACE TABLE `{0}.{1}.addresses` AS
        SELECT
          ROW_NUMBER() OVER() - 1 id,
          addr
        FROM (
          SELECT
            DISTINCT addr
          FROM (
            SELECT
              `to` AS addr
            FROM
              `{0}.{1}.{2}`
            UNION ALL
            SELECT
              `from` AS addr
            FROM
              `{0}.{1}.{2}` )
          ORDER BY
            addr )
        ORDER BY
          id""".format(bq_project,bq_dataset,bq_table)

'\n    CREATE OR REPLACE TABLE `wu-btcgraph.btc.addresses` AS\n        SELECT\n          ROW_NUMBER() OVER() - 1 id,\n          addr\n        FROM (\n          SELECT\n            DISTINCT addr\n          FROM (\n            SELECT\n              `to` AS addr\n            FROM\n              `wu-btcgraph.btc.bitcoin_transactions`\n            UNION ALL\n            SELECT\n              `from` AS addr\n            FROM\n              `wu-btcgraph.btc.bitcoin_transactions` )\n          ORDER BY\n            addr )\n        ORDER BY\n          id'